In [4]:
import os,sys

import pulp

import numpy as np

ROOTDIR = os.path.abspath(os.path.realpath('./')) + '/Py'

sys.path.append(os.path.join(ROOTDIR, ''))

from Columns_generation import Columns_generation

from Reposition import Reposition



In [5]:
'''State and Action'''

data_str='2019-11-01'

Load_path='./Data/Processed/'

Daily_path='./Data/Daily_Feature/'

State=np.load(os.path.join(Load_path,'State.npy'))

Action=np.load(os.path.join(Load_path,'Action.npy')).item()

Request_count_dic=np.load(os.path.join(Daily_path,'Request_count_dic'+data_str+'.npy')).item()

reposition=Reposition(State,Action,Request_count_dic)

# Initial Modleing

The optimization problem can be formulated as follows:

\begin{equation}
\begin{aligned}
\label{eq::Real-time_Optimization}
\max & \sum_{i}^{N}\sum_{a}^{L} y(i,t,s^{i},a) \cdot Q^{i}(t,s^{i},a)
\\
s.t. &  \sum_{i}^{N} y(i,t,s^{i},a) \leq \Delta_{t,a}, \forall a \in L\\
&\sum_{a}^{L} y(i,t,s^{i},a) \leq 1, \forall i \in N;
\\
& y(i,t,s,a) \in \{0,1\}
\\
\end{aligned}
\end{equation}

Restricted master problem:

\begin{equation}
\begin{aligned}
\label{eq::Restricted master problems}
\max & \sum_{a_{j}}^{L} \sum_{k}^{K_{a_{j}}} [\sum_{i}^{N} y^{k}(i,t,s^{i},a_{j}) \cdot Q^{i}(t,s^{i},a)] \cdot \lambda_{a_{j}}^{k}
\\
s.t. &  \sum_{a_{j}}^{L} \sum_{k}^{K_{a_{j}}} y^{k}(i,t,s^{i},a_{j}) \lambda_{a_{j}}^{k} \leq 1,  \forall i \in N\\
&\sum_{k}^{K_{a_{j}}} \lambda_{a_{j}}^{k} \leq 1, \forall a_{j} \in L;
\\
& \lambda_{a_{j}}^{k} \in \{0,1\}, \forall a_{j} \in L, \forall k \in K_{a_{j}};
\\
\end{aligned}
\end{equation}

Pricing problem:

\begin{equation}
\begin{aligned}
\label{eq::Pricing problem}
\max & \sum_{i}^{N}  y(i,t,s^{i},a_{j}) \cdot (Q^{i}(t,s^{i},a)-\mu_{i})\\
s.t. & \sum_{i}^{N} y(i,t,s^{i},a_{j}) \leq \Delta_{t,a_{j}}\\
& y(i,t,s^{i},a_{j}) \in \{0,1\} \\
\end{aligned}
\end{equation}




In [2]:
'''Example'''

'''ParaM'''

Driver_list=['d1','d2','d3']

Destination_list=['g1','g2','g3','g4','g5']

Q_table={'d1':{'g1':5,'g2':2,'g3':3,'g4':7,'g5':5},\
       'd2':{'g1':3,'g2':5,'g3':3,'g4':8,'g5':100},\
       'd3':{'g1':40,'g2':1,'g3':2,'g4':9,'g5':5}}


Capacity_={'g1':1,'g2':1,'g3':1,'g4':1,'g5':1}


"""
**********************************************************************************

**********************************************************************************

**********************************************************************************

**********************************************************************************

**********************************************************************************

"""


    

'''Testing'''

CG=Columns_generation(Driver_list,Destination_list,Q_table,Capacity_)

'''Generate initialized solutions'''

Solution,K_num=CG.Get_initialization()


'''Columns Generation'''

count=0


Any_Pricing_Obj=0.01

while Any_Pricing_Obj>0:

    Any_Pricing_Obj=0
    
    count+=1
    
    '''Solve the RMP'''
    
    Lambda,RMP=CG.Get_RMP(Solution,K_num)

    RMP.solve()
    
    Rep_action=CG.Get_solution(Lambda,Solution)

    Dual=[RMP.constraints[i].pi for i in RMP.constraints]

    print("*"*30)

    print("{0}th Optimal objective of RMP".format(count),pulp.value(RMP.objective))
    
    print("{0}th Soulution of RMP".format(count),Rep_action)
    
    print("*"*30)
    
    '''Construct the Pricing problem'''

    for a in Destination_list:

        Y,Pricing=CG.Get_Pricing(Dual,a)

        Pricing.solve()
        
        if pulp.value(Pricing.objective):

            if pulp.value(Pricing.objective)>0:

                Any_Pricing_Obj=pulp.value(Pricing.objective)

                '''Generate new columns'''

                Y_=np.array([[Y[var].varValue for var in Y]],dtype='int')

                Solution[a]=np.append(Solution[a],Y_,axis=0)

                K_num[a]+=1

                break

******************************
1th Optimal objective of RMP 7.0
1th Soulution of RMP {'d1': 'g4'}
******************************
******************************
2th Optimal objective of RMP 47.0
2th Soulution of RMP {'d3': 'g1', 'd1': 'g4'}
******************************
******************************
3th Optimal objective of RMP 52.0
3th Soulution of RMP {'d3': 'g1', 'd2': 'g2', 'd1': 'g4'}
******************************
******************************
4th Optimal objective of RMP 52.0
4th Soulution of RMP {'d3': 'g1', 'd2': 'g2', 'd1': 'g4'}
******************************
******************************
5th Optimal objective of RMP 52.0
5th Soulution of RMP {'d3': 'g1', 'd2': 'g2', 'd1': 'g4'}
******************************
******************************
6th Optimal objective of RMP 53.0
6th Soulution of RMP {'d3': 'g1', 'd2': 'g4', 'd1': 'g5'}
******************************
******************************
7th Optimal objective of RMP 147.0
7th Soulution of RMP {'d3': 'g1', 'd1': 'g4', '

In [6]:
'''Test demo'''

Rep_action=reposition.MILP_Column_Generation(Driver_list,Destination_list,Q_table,Capacity_)

Rep_action

{'d3': 'g1', 'd1': 'g4', 'd2': 'g5'}